In [1]:
from SI_sorting import run_prepro_and_sorting
from SI_tools import clear_SItempdir

############################################################################################################################################################################################################
#                                                          Set the Parent Directories
############################################################################################################################################################################################################

#########################################################################################################################
# "parentRecordingFolder"
#   Folder from where to read NWB (read from server, unless you copied the files into the local machine)
parentRecordingFolder = 'Y:\Data_Albus\Albus_NeuralData\Albus-S1'

#########################################################################################################################
# "parentPreproFolder":
#   Folder to save all the Plots & recording objects from the preprocessing (server is recommended to keep record of the sorting process)
parentPreproFolder = 'Y:\Data_Albus\Albus_NeuralData\Albus-S1-Prepro' 

#########################################################################################################################
# "parentSortingFolder" 
# Folder to save "recording" (binalry) & "sorting_analyzer"
# if "parentSortingFolder" is not provided, then:
#       if exporting "sorting_analyzer" to phy is True, both objects will be created temporary and deleted at the end
#       if exporting "sorting_analyzer" to phy is False, both objects will be created in the same parentPreproFolder
parentSortingFolder = None

#########################################################################################################################
# "parentPhyFolder":
# Folder to save the results from sorting into phy format.
# If None, it will use the same path as the "parentPreproFolder"
parentPhyFolder = None 

######################################################################################################
# "localProcess_NWB" : 
#       if TRUE, it will copy NWB files into a temporary directory in the same disk as this script and it will deleted the NWB files at the end
#                it will not create recording objects due to the lost of path links after deleting the temprary files
#       if FALSE it will read files from original location and it will create recording objects with paths linked to the original NWB locations.
localProcess_NWB = False

# "rewrite_prepro" : 
#       if TRUE it will run all the preprocessing regardless of finding the previous recording objects
#       if FALSE it will check whether preprocessed and motion_iterpolated recordings exist 
#           if those exists, it will be reloaded and pass it to the sorter
#           if those not exists then it will re run the preprocessing and pass it to the sorter
rewrite_prepro = False

# "export2phy"
#       if FALSE it will save the sorter analyzer and the binary recording folder (see "parentSortingFolder")
#       if TRUE it will export the sorter analyzer to phy with the binary recording. The sorter analyzer might be deleted depending on the value of "parentSortingFolder"
export2phy = True

In [2]:
############################################################################################################################################################################################################
#                                               SELECT EXPERIMENT DAY AND SESSIONS (listID or None)
############################################################################################################################################################################################################

#########################################################################################################################
# Experiment Day : YEAR, MONTH, DAY
sessionYear = 2024
sessionMonth = 2
sessionDay = 23

#########################################################################################################################
# ElectrodeGroupe Name (based on NWBfile naming schema) 
# Note: if there were different coordinates within the same day, you need to add '-sub1', '-sub2', etc. 
# It is recommended to run "SI_run_prepro" first, it will create folder for all the unique electrode groups from that day.
# In case you want to explore how many electrodeGroup-subX exists, run:
#       from SI_tools import print_recordingInfo
#       print_recordingInfo(parentRecordingFolder, sessionYear, sessionMonth, sessionDay)
###########################################################################################################################
electrodeGroup_Name = 'raw-B1-129-160' 

#########################################################################################################################
# Session index is based on the dateTime. Zero is the first session of the day from the selected electrodeGroup
# If None, it will use all sessions from this electrodeGroup in this experiment Day
session_index = [0, 1] 

In [3]:
############################################################################################################################################################################################################
#                                                                   SET GENERAL PARAMETERS FOR PREPROCESSING
############################################################################################################################################################################################################

######################################################################################################
# PARAMETER FOR "LOCAL REFERENCING"
exclude_radius_chans = 1 # Number of neighbor channels to exclude because are too close to the reference channel
include_radius_chans = 4 # Number of neighbor channels delineates the outer boundary of the annulus whose role is to exclude channels that are too far away

######################################################################################################
# NOISY FREQUENCY TO BE REMOVED (i.e., "spre.notch_filter")
noisy_freq = None

# TODO: bandpass filter
###########################################################
# DETECT PEAKS: 
peak_detect_threshold = 5 #  MAD: Median Amplitude Deviations
peak_sign = 'both' # (“neg” | “pos” | “both”)
nearest_chans = 3 # Number of neighbor channels to search for events that occurred at the same time

######################################################################################################
# GENERAL PARAMETER FOR "PEAKS LOCATIONS"
ms_before = 0.6
ms_after = 1.5

######################################################################################################
# MOTION ESTIMATION
do_motion = True # if TRUE it will do motion estimation and interpolation

# OPTIONS to run : monopolar triangulation + non-rigid + decentralized + kriging
motion_rigid = False
motion_options = {
    'method': 'dredge_ap', # Paninski Lab
    'method_kwargs' : {} 
}
interpolate_options = {
    'method': 'kriging', # Kilosort-like
    'border_mode': 'remove_channels' # ('remove_channels' | 'force_extrapolate' | 'force_zeros')
}

In [4]:
############################################################################################################################################################################################################
#                                                                   SET GENERAL PARAMETERS FOR SORTING
############################################################################################################################################################################################################

# If False, it will only run preprocessing (If you changed some preprocessing params, set "rewrite_prepro" to True)
run_sorting = True

######################################################################################################
# SORTER NAME (options: mountainsort5 & kilosort4)
sorterName = 'kilosort4'

# mountainsort5: ZCA-Whitening units (default = 5.5) [you can try with values as low as 2.5]
# kilosort4: STD units from whithening traces (default = 3) [you can try values as low as 1.5]
# If None, it will use default values according to each sorting
sorter_detect_threshold = 3

# Number of neighbor channels to search for the same waveform during sorting & sorting_analyzer
# "nearest_chans = 0" will perform similar to sorting single channels. 
# If set to None, it will use the "nearest_chans" value from peaks locations
sorter_nearest_chans = 2 

In [ ]:
run_prepro_and_sorting(
    parentRecordingFolder = parentRecordingFolder, 
    parentPreproFolder = parentPreproFolder, 
    parentSortingFolder = parentSortingFolder, 
    parentPhyFolder = parentPhyFolder, 
    sessionYear = sessionYear, 
    sessionMonth = sessionMonth, 
    sessionDay = sessionDay, 
    electrodeGroup_Name = electrodeGroup_Name, 
    session_index = session_index,
    local_radius_chans = (exclude_radius_chans, include_radius_chans), 
    noisy_freq = noisy_freq, 
    ms_before = ms_before, 
    ms_after = ms_after, 
    peak_sign = peak_sign,
    nearest_chans = nearest_chans, 
    peak_detect_threshold = peak_detect_threshold, 
    do_motion = do_motion,
    motion_rigid = motion_rigid, 
    motion_options = motion_options, 
    interpolate_options = interpolate_options,
    localProcess_NWB = localProcess_NWB,
    rewrite_prepro = rewrite_prepro,
    run_sorting = run_sorting,
    sorterName = sorterName, 
    sorter_detect_threshold = sorter_detect_threshold,
    sorter_nearest_chans = sorter_nearest_chans,
    export2phy = export2phy
)

clear_SItempdir()